## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,2020-08-02 18:22:52,-33.93,18.42,61.00,87,0,10.29,clear sky
1,1,Safaga,EG,2020-08-02 18:26:26,26.73,33.94,89.60,55,11,2.24,few clouds
2,2,Saint-Philippe,RE,2020-08-02 18:24:56,-21.36,55.77,69.80,60,90,12.75,overcast clouds
3,3,Oberwart,AT,2020-08-02 18:26:27,47.29,16.21,73.00,79,100,4.61,overcast clouds
4,4,Bluff,NZ,2020-08-02 18:24:47,-46.60,168.33,46.49,94,71,5.32,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Safaga,EG,2020-08-02 18:26:26,26.73,33.94,89.60,55,11,2.24,few clouds
5,5,Albany,US,2020-08-02 18:20:59,42.60,-73.97,79.00,67,70,4.00,broken clouds
18,18,George Town,MY,2020-08-02 18:22:23,5.41,100.34,84.20,79,20,4.70,few clouds
19,19,Westlock,CA,2020-08-02 18:26:30,54.15,-113.87,78.80,53,1,9.17,clear sky
21,21,Arraial Do Cabo,BR,2020-08-02 18:24:59,-22.97,-42.03,75.20,53,40,10.29,scattered clouds
22,22,Carutapera,BR,2020-08-02 18:21:50,-1.20,-46.02,85.08,64,23,8.16,light rain
23,23,Redmond,US,2020-08-02 18:21:51,47.67,-122.12,75.00,68,8,1.01,clear sky
25,25,Beaufort,MY,2020-08-02 18:26:31,5.35,115.75,80.01,100,20,2.48,few clouds
31,31,Katsuura,JP,2020-08-02 18:25:34,35.13,140.30,75.20,88,20,4.70,few clouds
42,42,Kapaa,US,2020-08-02 18:24:42,22.08,-159.32,80.60,69,1,11.41,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                239
City                   239
Country                239
Date                   239
Lat                    239
Lng                    239
Max Temp               239
Humidity               239
Cloudiness             239
Wind Speed             239
Current Description    239
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
preferred_cities_df.notnull().sum()

City_ID                239
City                   239
Country                239
Date                   239
Lat                    239
Lng                    239
Max Temp               239
Humidity               239
Cloudiness             239
Wind Speed             239
Current Description    239
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = "np.nan"
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Safaga,EG,89.60,few clouds,26.73,33.94,np.nan
5,Albany,US,79.00,broken clouds,42.60,-73.97,np.nan
18,George Town,MY,84.20,few clouds,5.41,100.34,np.nan
19,Westlock,CA,78.80,clear sky,54.15,-113.87,np.nan
21,Arraial Do Cabo,BR,75.20,scattered clouds,-22.97,-42.03,np.nan
22,Carutapera,BR,85.08,light rain,-1.20,-46.02,np.nan
23,Redmond,US,75.00,clear sky,47.67,-122.12,np.nan
25,Beaufort,MY,80.01,few clouds,5.35,115.75,np.nan
31,Katsuura,JP,75.20,few clouds,35.13,140.30,np.nan
42,Kapaa,US,80.60,clear sky,22.08,-159.32,np.nan


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Safaga,EG,89.60,few clouds,26.73,33.94,Lamar Resort Abu Soma
5,Albany,US,79.00,broken clouds,42.60,-73.97,np.nan
18,George Town,MY,84.20,few clouds,5.41,100.34,Cititel Penang
19,Westlock,CA,78.80,clear sky,54.15,-113.87,Ramada by Wyndham Westlock
21,Arraial Do Cabo,BR,75.20,scattered clouds,-22.97,-42.03,Pousada Porto Praia
22,Carutapera,BR,85.08,light rain,-1.20,-46.02,Lidera
23,Redmond,US,75.00,clear sky,47.67,-122.12,Seattle Marriott Redmond
25,Beaufort,MY,80.01,few clouds,5.35,115.75,The Gem Hotel Beaufort
31,Katsuura,JP,75.20,few clouds,35.13,140.30,Katsuura Hotel Mikazuki
42,Kapaa,US,80.60,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach


In [12]:
hotel_df.count()

City                   239
Country                239
Max Temp               239
Current Description    239
Lat                    239
Lng                    239
Hotel Name             239
dtype: int64

In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

In [17]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))